# Performance of quantization on the RGB domain

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.axes as ax
import pylab
import math
import numpy as np
from scipy import signal
import cv2
import os
!pwd
!ln -sf ~/MRVC/src/deadzone.py .
!ln -sf ~/MRVC/src/frame.py .
import deadzone
import frame

In [ ]:
def __quantizer(x, quantization_step):
    k = (x / quantization_step).astype(np.int16)
    return k

def __dequantizer(k, quantization_step):
    y = quantization_step * k
    return y

def q_deq(x, quantization_step):
    k = deadzone.quantize(x, quantization_step)
    y = deadzone.dequantize(k, quantization_step)
    return k, y

# Notice that, although this is a dead-zone quantizer, we are not going
# to work with negative samples, and therefore, the dead-zone
# does not have any effect.

In [ ]:
def __load_frame(prefix):
    fn = f"{prefix}.png"
    frame = cv2.imread(fn, cv2.IMREAD_UNCHANGED) # [rows, cols, comp]
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = np.array(frame)
    frame = frame.astype(np.float32) - 32768.0
    frame = frame.astype(np.uint8)
    return frame

def __write_frame(frame, prefix):
    frame = frame.astype(np.float32)
    frame += 32768.0
    frame = frame.astype(np.uint16)
    cv2.imwrite(f"{prefix}.png", frame)

In [ ]:
def load_indexes(prefix):
    #load_frame(prefix)
    frame.load(prefix)
    
def write_indexes(prefix):
    #write_frame(prefix)
    #frame.write(prefix)
    frame.save(prefix)

In [ ]:
fn = "/home/vruiz/MRVC/sequences/stockholm_5_frames/000"
_frame = frame.load(fn)
print(_frame.max(), _frame.min())

In [ ]:
def __normalize(img):
    max_component = np.max(img)
    min_component = np.min(img)
    max_min_component = max_component - min_component
    return (img-min_component)/max_min_component

def merge(frame):
    return cv2.merge((frame[0], frame[1], frame[2]))

def show(_frame, prefix):
    _frame = frame.normalize(_frame)
    plt.figure(figsize=(10,10))
    plt.title(prefix, fontsize=20)
    plt.imshow(merge(_frame))

In [ ]:
show(_frame, fn)

In [ ]:
def average_energy(x):
    return np.sum(x.astype(np.double)*x.astype(np.double))/(np.size(x))

def MSE(x, y):
    error_signal = x - y
    return average_energy(error_signal)

def RMSE(x, y):
    error_signal = x - y
    return math.sqrt(MSE(error_signal))

In [ ]:
def bytes_per_frame(_frame):
    frame.save(_frame, "/tmp/frame")
    length_in_bytes = os.path.getsize("/tmp/frame.png")
    return length_in_bytes

def bytes_per_grayframe(_frame):
    cv2.imwrite("/tmp/frame.png", _frame)
    length_in_bytes = os.path.getsize("/tmp/frame.png")
    return length_in_bytes

def entropy_in_bits_per_symbol(sequence_of_symbols):
    value, counts = np.unique(sequence_of_symbols, return_counts = True)
    probs = counts / len(sequence_of_symbols)
    n_classes = np.count_nonzero(probs)

    if n_classes <= 1:
        return 0

    entropy = 0.
    for i in probs:
        entropy -= i * math.log(i, 2)

    return entropy

## RD curve using same $\Delta$ for each RGB channel
To see the contribution of each channel to the RD curve.

In [ ]:
def RD_curve(x):
    points = []
    for q_step in range(0, 8):
        k, y = q_deq(x, 1<<q_step)
        #print(k.max())
        rate = bytes_per_frame(k)
        distortion = MSE(x, y)
        points.append((rate, distortion))
        print(f"q_step={1<<q_step:>3}, rate={rate:>7} bytes, distortion={distortion:>6.1f}")
    return points

RD_points = RD_curve(_frame)

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*RD_points), c='m', marker="o")
pylab.title("RD of a RGB image")
pylab.xlabel("Bytes/Frame")
pylab.ylabel("MSE")
pylab.show()

In [ ]:
k, y = q_deq(_frame, 64)
show(y, "")

## RD curves of each channel

In [ ]:
def only_R_RD_curve(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        #R_frame = RGB_frame[:,:,0]
        R_frame = RGB_frame[0]
        #dequantized_R_frame = np.empty_like(RGB_frame)
        #k = np.empty_like(RGB_frame)
        k, dequantized_R_frame = q_deq(R_frame, 1<<q_step)
        #dequantized_R_frame[0,:,:] = _dequantized_R_frame[:,:]
        #k[0,:,:] = _k[:,:]
        rate = bytes_per_grayframe(k)
        distortion = MSE(R_frame, dequantized_R_frame)
        RD_points.append((rate, distortion))
        print(f"q_step={1<<q_step:>3}, rate={rate:>7} bytes, distortion={distortion:>6.1f}")
    return RD_points

def only_G_RD_curve(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        #G_frame = RGB_frame[:,:,1]
        G_frame = RGB_frame[1]
        #dequantized_G_frame = np.empty_like(RGB_frame)
        #k = np.empty_like(RGB_frame)
        k, dequantized_G_frame = q_deq(G_frame, 1<<q_step)
        #dequantized_G_frame[1,:,:] = _dequantized_G_frame[:,:]
        #k[1,:,:] = _k[:,:]        
        rate = bytes_per_grayframe(k)
        distortion = MSE(G_frame, dequantized_G_frame)
        RD_points.append((rate, distortion))
        print(f"q_step={1<<q_step:>3}, rate={rate:>7} bytes, distortion={distortion:>6.1f}")
    return RD_points

def only_B_RD_curve(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        #B_frame = RGB_frame[:,:,2]
        B_frame = RGB_frame[2]
        #dequantized_B_frame = np.empty_like(RGB_frame)
        #k = np.empty_like(RGB_frame)
        k, dequantized_B_frame = q_deq(B_frame, 1<<q_step)
        #dequantized_B_frame[2,:,:] = _dequantized_B_frame[:,:]
        #k[2,:,:] = _k[:,:] 
        rate = bytes_per_grayframe(k)
        distortion = MSE(B_frame, dequantized_B_frame)
        RD_points.append((rate, distortion))
        print(f"q_step={1<<q_step:>3}, rate={rate:>7} bytes, distortion={distortion:>6.1f}")
    return RD_points

only_R_points = only_R_RD_curve(_frame)
only_G_points = only_G_RD_curve(_frame)
only_B_points = only_B_RD_curve(_frame)

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*RD_points), c='m', marker="x",
           label='$\Delta_{\mathrm{R}} = \Delta_{\mathrm{G}} = \Delta_{\mathrm{B}}$')
pylab.plot(*zip(*only_R_points), c='r', marker="o",
           label='Only R')              
pylab.plot(*zip(*only_G_points), c='g', marker="o",
           label='Only G')              
pylab.plot(*zip(*only_B_points), c='b', marker="o",
           label='Only B')              
pylab.title("RD Performance")
pylab.xlabel("Bytes/Frame")
pylab.ylabel("MSE")
plt.legend(loc='upper right')
pylab.show()

The $\Delta_{\mathrm{R}} = \Delta_{\mathrm{G}} = \Delta_{\mathrm{B}}$ quantization scheme is near optimal because the slope at the different quantization points is almost the same. This can be seen in the next experiment

## Testing a different quantization configuration

In [ ]:
N = 6
def only_R_RD_curve(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        k_R, dequantized_RGB_frame_R = q_deq(RGB_frame[0], 1<<q_step)
        k_G, dequantized_RGB_frame_G = q_deq(RGB_frame[1], 1<<N)
        k_B, dequantized_RGB_frame_B = q_deq(RGB_frame[2], 1<<N)
        k = np.array([k_R, k_G, k_B])
        dequantized_RGB_frame = np.array([dequantized_RGB_frame_R, dequantized_RGB_frame_G, dequantized_RGB_frame_B])
        rate = bytes_per_frame(k)
        distortion = MSE(RGB_frame, dequantized_RGB_frame)
        RD_points.append((rate, distortion))
        print(f"q_step={1<<q_step:>3}, rate={rate:>7} bytes, distortion={distortion:>6.1f}")
    return RD_points

def only_G_RD_curve(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        k_R, dequantized_RGB_frame_R = q_deq(RGB_frame[0], 1<<N)
        k_G, dequantized_RGB_frame_G = q_deq(RGB_frame[1], 1<<q_step)
        k_B, dequantized_RGB_frame_B = q_deq(RGB_frame[2], 1<<N)
        k = np.array([k_R, k_G, k_B])
        dequantized_RGB_frame = np.array([dequantized_RGB_frame_R, dequantized_RGB_frame_G, dequantized_RGB_frame_B])
        rate = bytes_per_frame(k)
        distortion = MSE(RGB_frame, dequantized_RGB_frame)
        RD_points.append((rate, distortion))
        print(f"q_step={1<<q_step:>3}, rate={rate:>7} bytes, distortion={distortion:>6.1f}")
    return RD_points

def only_B_RD_curve(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        k_R, dequantized_RGB_frame_R = q_deq(RGB_frame[0], 1<<N)
        k_G, dequantized_RGB_frame_G = q_deq(RGB_frame[1], 1<<N)
        k_B, dequantized_RGB_frame_B = q_deq(RGB_frame[2], 1<<q_step)
        k = np.array([k_R, k_G, k_B])
        dequantized_RGB_frame = np.array([dequantized_RGB_frame_R, dequantized_RGB_frame_G, dequantized_RGB_frame_B])
        rate = bytes_per_frame(k)
        distortion = MSE(RGB_frame, dequantized_RGB_frame)
        RD_points.append((rate, distortion))
        print(f"q_step={1<<q_step:>3}, rate={rate:>7} bytes, distortion={distortion:>6.1f}")
    return RD_points

only_R_points = only_R_RD_curve(_frame)
only_G_points = only_G_RD_curve(_frame)
only_B_points = only_B_RD_curve(_frame)

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*RD_points), c='m', marker="x",
           label='$\Delta_{\mathrm{R}} = \Delta_{\mathrm{G}} = \Delta_{\mathrm{B}}$')
pylab.plot(*zip(*only_R_points), c='r', marker="o",
           label='$\Delta_{\mathrm{R}}~\mathrm{varies},~\Delta_{\mathrm{G}}=\Delta_{\mathrm{B}}=$' + '{}'.format(1<<N))              
pylab.plot(*zip(*only_G_points), c='g', marker="o",
           label='$\Delta_{\mathrm{G}}~\mathrm{varies},~\Delta_{\mathrm{R}}=\Delta_{\mathrm{B}}=$' + '{}'.format(1<<N))              
pylab.plot(*zip(*only_B_points), c='b', marker="o",
           label='$\Delta_{\mathrm{B}}~\mathrm{varies},~\Delta_{\mathrm{R}}=\Delta_{\mathrm{G}}=$' + '{}'.format(1<<N))              
pylab.title("RD Performance")
pylab.xlabel("Bytes/Frame")
pylab.ylabel("MSE")
plt.legend(loc='upper right')
pylab.show()

As it can be seen, the best configuration matches $\Delta_{\mathrm{R}} = \Delta_{\mathrm{G}} = \Delta_{\mathrm{B}}$.